# Vectors and embedding

The notebook is a step-by-step guide to processing documents and creating vector representations (embeddings) of their content. Here's a simplified explanation of what it does:

1) **Load the Document**: It starts by loading a document (e.g., a PDF file) into the program.
2) **Split the Document**: The document is then split into smaller chunks to make it easier to process.
3) **Create Embeddings**: For each chunk, the notebook uses a model to create a vector representation (embedding). This is like converting the text into a series of numbers that capture its meaning.
4) **Store the Embeddings**: These embeddings are stored in a special database called a vector store. This allows for efficient searching and retrieval based on the content of the documents.
5) **Query the Vector Store**: Finally, the notebook demonstrates how to query this vector store to find relevant document chunks based on a search query.

Overall, the notebook shows how to transform text documents into a format that can be easily searched and analyzed using machine learning techniques.

# Qdrant
Qdrant is a high-performance vector database used to store and search vector representations (embeddings) of data efficiently. It is scalable and integrates well with machine learning and AI applications, making it suitable for production environments.

In [6]:
# https://python.langchain.com/docs/tutorials/retrievers/
import os
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http import models

In [7]:
OLLAMA_SERVER = os.getenv("OLLAMA_SERVER")

In [8]:
# 1. We load the document
# -----------------------
file_path = "./docs/test.pdf"
loader    = PyPDFLoader(file_path)
docs      = loader.load()
print(len(docs))

7


In [9]:
print(f"{docs[0].page_content[:100]}\n")
docs[0].metadata

du Code judiciaire.  Si la créance garantie n'est pas encore exigible, le créancier gagiste verse le



{'source': './docs/test.pdf', 'page': 0}

In [10]:
# We split this document un chunks
# --------------------------------
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=10, 
    add_start_index=True
)
all_splits    = text_splitter.split_documents(docs)
len(all_splits)

79

In [11]:
# We select the embedder and create vectors
# ------------------------------------------
embeddings = OllamaEmbeddings(
    base_url=OLLAMA_SERVER, 
    model="mxbai-embed-large"
)
vectors    = [embeddings.embed_query(text.page_content) for text in all_splits]

In [12]:
# Instantiate the QDrant client and recreate a collection
#--------------------------------------------------------
qdrant_client     = QdrantClient(host='lawboxai_qdrant')
qdrant_collection = 'example'
qdrant_client.delete_collection(collection_name=qdrant_collection)

True

In [13]:
qdrant_client.create_collection(
   collection_name=qdrant_collection,
   vectors_config=models.VectorParams(
       size=len(vectors[0]), 
       distance=models.Distance.COSINE
    ),
)
vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=qdrant_collection,
    embedding=embeddings
)
vector_store.add_documents(all_splits)
query = "Les défauts de payement"

In [14]:
# Similarity_search: Returns the top documents based solely on vector similarity
docs  = vector_store.similarity_search(query, k=10,)
for doc in docs:
    print(50*'-')
    print(doc.page_content)

--------------------------------------------------
Art. 8.37. Force probante  Le serment ne fait preuve qu'au profit de celui qui l'a déféré et de ses héritiers et ayants cause, ou contre eux.  Le serment déféré par l'un des créanciers solidaires au débiteur ne libère celui-ci que pour la part de cecréancier.  Le serment déféré au débiteur principal libère également les cautions.  Le serment déféré à l'un des débiteurs solidaires profite aux codébiteurs.  Le serment déféré à la caution profite au débiteur principal.  Dans ces deux derniers
--------------------------------------------------
derniers cas, le serment du codébiteur solidaire ou de la caution ne profite aux autrescodébiteurs ou au débiteur principal que lorsqu'il a été déféré sur la dette principale, et non sur le fait de lasolidarité ou du cautionnement.
--------------------------------------------------
Art. 69.Ecrit  Des biens meubles vendus avec une clause suspendant le transfert de propriété jusqu'au paiement intégral 

In [15]:
# Similar to similarity_search, but also returns relevance scores
docs  = await vector_store.asimilarity_search_with_relevance_scores(query,k=5)   #.similarity_search(query, k=10,)
for doc in docs:
    print(50*'-',doc[1])
    print(doc[0].page_content)

-------------------------------------------------- 0.8693486500000001
Art. 8.37. Force probante  Le serment ne fait preuve qu'au profit de celui qui l'a déféré et de ses héritiers et ayants cause, ou contre eux.  Le serment déféré par l'un des créanciers solidaires au débiteur ne libère celui-ci que pour la part de cecréancier.  Le serment déféré au débiteur principal libère également les cautions.  Le serment déféré à l'un des débiteurs solidaires profite aux codébiteurs.  Le serment déféré à la caution profite au débiteur principal.  Dans ces deux derniers
-------------------------------------------------- 0.86478035
derniers cas, le serment du codébiteur solidaire ou de la caution ne profite aux autrescodébiteurs ou au débiteur principal que lorsqu'il a été déféré sur la dette principale, et non sur le fait de lasolidarité ou du cautionnement.
-------------------------------------------------- 0.85686935
Art. 69.Ecrit  Des biens meubles vendus avec une clause suspendant le transfert

In [16]:
# The LangChain way, using Retriever
retriever = vector_store.as_retriever()
query = "Les défauts de payement"
retrieved_docs = retriever.invoke(query,k=7)

# Afficher les résultats
for doc in retrieved_docs:
    print(50*'-')
    print(doc.page_content)

--------------------------------------------------
Art. 8.37. Force probante  Le serment ne fait preuve qu'au profit de celui qui l'a déféré et de ses héritiers et ayants cause, ou contre eux.  Le serment déféré par l'un des créanciers solidaires au débiteur ne libère celui-ci que pour la part de cecréancier.  Le serment déféré au débiteur principal libère également les cautions.  Le serment déféré à l'un des débiteurs solidaires profite aux codébiteurs.  Le serment déféré à la caution profite au débiteur principal.  Dans ces deux derniers
--------------------------------------------------
derniers cas, le serment du codébiteur solidaire ou de la caution ne profite aux autrescodébiteurs ou au débiteur principal que lorsqu'il a été déféré sur la dette principale, et non sur le fait de lasolidarité ou du cautionnement.
--------------------------------------------------
Art. 69.Ecrit  Des biens meubles vendus avec une clause suspendant le transfert de propriété jusqu'au paiement intégral 